# Results
The idea of this project is to understand and implement different reinforcement learning algorithms and compare their performance. Initially, we start with the following four algorithms:
1. **Policy-Gradient:** Current states are the input, categorical actions the output. During training the targets are given as the action chosen, multiplied by $\pm 1$ depending on the reward. This works due to the structure of the softmax activation function and  categorical-crossentropy loss function.
2. **Deep-Q-Learner:** The inputs are given as the current state & and a candidate action. The output is the so-called Q-value — an estimate of the quality of an action. For action-selection we choose the action with the highest Q-value. The targets during training are formed by using the algorithm‘s *Bellman eq.*: $$Q(s_t, a_t) =r_t + \gamma \cdot \mathrm{max}_a Q(s_{t+1}, a_{t+1})$$ where we denote by $\gamma$ the so called *discount factor* which controls to which the model attends to long term behaviour.
3. **Deep-Q-Learner with Target Network:** This is an improved version of the former algorithm in which we build the targets during learning from a second so-called *target network*. This network is not trained, but instead we copy the *training network‘s* weights $\theta_\mathrm{train}$ every $d$ episodes using $$\theta_{\mathrm{target}} \leftarrow \tau \cdot \theta_{\mathrm{train}} + (1 - \tau) \cdot \theta_{\mathrm{target}}$$ for some $\tau \in [0,1]$. The intuition is, that we make convergence more stable by keeping the training targets more stable.
4. **Double-Q-Learner:** Once again this is an iteration on the former two algorithms. The difference is that we use a modified Bellman equation $$Q_{\mathrm{train}}(s_t, a_t)  = r_t + \gamma \cdot Q_{\mathrm{target}} \left( s_{t+1}, \mathrm{argmax}_a Q_{\mathrm{train}}(s_{t+1}, a_{t+1}) \right)$$ in which we have split the action-selection (done by the training network) from the action-evaluation (done by the target network). The idea behind this is to mitigate the overestimation problem of the Q-function during training (see [*van Hasselt, et al. (2015)*](https://arxiv.org/abs/1509.06461) for details).

Since the focus of this project is on learning and model-comparison, we start with a very simple environment from the [OpenAI-Gym, called CartPole-v1](https://gymnasium.farama.org/environments/classic_control/cart_pole/). However, all of the code in this repository has been kept completely generic to be able to employ our algorithms on  different environments later on. For now, we want to be able to stick to small networks and make sure all the algorithms work as expected and be able to compare the different models quickly.

Before we get to the code, please use the following two cells to install the required packages and to import the necessary modules, if you want to run the later cells.

In [ ]:
!pip install gymnasium numpy plotly scipy tensorflow

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"

from agents import ModelHyperParams, PolicyGradient, QLearner, RandomAgent
from environment_handler import CartPoleHandler
from scripts import parameter_scan, train, TrainingHyperParams

As an example on how to use this code base to train our version of the Q-Learner $\textcolor{red}{\text{choose best algo}}$ on this particular problem we could run the following code:

In [ ]:
# TODO Adjust hyper parameters later on.
model_params = ModelHyperParams(
    initial_epsilon=0.5,
    epsilon_decay_constant=0.7,
    gamma=0.5,
    no_hidden_layers=2,
    units_per_hidden_layer=12,
    kernel_initializer="he_uniform"
)
training_params = TrainingHyperParams(
    batch_size=256,
    epochs=1000,
    steps_per_epoch=2560,
    max_buffer_size=256000
)

env_handler = CartPoleHandler()
agent = QLearner(env_handler, model_params)

result = train(env_handler, agent, training_params)

Of course getting the model to converge depends very strongly on the hyperparameters chosen. This means we need to do some hyperparameter tuning before we can get to comparing the different algorithms.

## Hyperparameter Tuning
In principle the different algorithms depend on multiple hyperparameters which a priori cannot be assumed to be independent or optimized separately. Moreover, training neural networks is not deterministic and reinforcement learning suffers even more from this problem. This means that we will have to run multiple training cycles for each hyperparameter configuration to actually have statistically significant differences. In the setting presented here we have mainly the following hyperparameters to adjust:
* **Batch size:** Due to the very noisy data in reinforcement learning this becomes even more important
* **Discount Factor $\gamma$:** For the Q-Learner algorithms this governs how much we take long-term behaviour into account (see Bellman eq.).
* **Proportion of random actions $\varepsilon$ & its decay:** All algorithms use an $\varepsilon$-greedy policy to balance exploration & exploitation.
* **Model architecture:** Number of layers and nodes per layer.
* **Optimiser & Learning Rate:** Do we choose adaptive optimisers such as *Adam*/*RMSProp* or static ones such as *SGD*. How does the *learning rate* affect convergence?
* **Weight initialiser:** Since convergence has been shown to be somewhat flaky, clearly the random initialisation of the weights matters and this is another parameter to explore.
* **Regularisation:** Since we might choose larger networks due to initialisation issues, we can consider counter-balancing this with regularisation (e.g. *L1*).
* **Target network update frequency $d$ and proportion $\tau$:** For those Q-Learners with target networks these two variables control how fast and abrupt we update the target network.

Clearly, it is unfeasable even for this simple environment to scan the whole hyper-parameter space. So what can we do? We can argue that some of the parameters are in good approximation independent of the others and in fact independent of the models as they are for the most part defined by the problem:
1. The optimal batch size is determined by how much data we need in a given batch to mitigate for the noise. While this will of course depend on the randomness of the policy and thereby the model its parameters, we suspect that these are second order effects and that the largest contribution is defined by the problem itself. This allows us to find the optimal batch size without varying the other variables.
2. The discount factor $\gamma$ encodes how long range behaviour affects the current reward and is therefore also mostly determined by the environment rather than the model.
3. The remaining parameters are tougher to separate as they are interlinked.

So how can we find e.g. the best batch size? We can run multiple training cycles for each batch size and count how many times the model converged to a successful state (here this is defined to be able to control the cart-pole for 200 subsequent actions before it falls). After each new training cycle we can compute a $p$-value of how likely the results come from different distributions using a *permutation test*. Once we have established the best batch size with a *p*-value of 95%, we assume this to be the best parameter setting. For demonstration purposes, we present here the code to run a parameter search, but of course it runs for a long time such that it should normally be run as a script in the background (and with more different batch sizes).

In [ ]:
ALLOWED_EPOCHS_TO_TRAIN = 10


def get_training_params(batch_size: int) -> TrainingHyperParams:
    return TrainingHyperParams(
        batch_size=batch_size,
        epochs=ALLOWED_EPOCHS_TO_TRAIN,
        steps_per_epoch=100*1024,
        max_buffer_size=10*100*1024
    )


make_agent_fct=lambda e, p: PolicyGradient(e, p, verbose=0)
make_env_handler_fct=lambda: CartPoleHandler()
model_params = ModelHyperParams(
    initial_epsilon=0.5,
    epsilon_decay_constant=0.7,
    no_hidden_layers=2,
    units_per_hidden_layer=24,
    optimizer="adam",
    kernel_initializer="he_uniform"
)
batch_sizes = [64, 256, 1024]
hyper_param_dict = {b: (get_training_params(b), model_params) for b in batch_sizes}

parameter_scan(hyper_param_dict, make_agent_fct, make_env_handler_fct, verbose=0)

In order to not clutter the repository with these files, we have not included the scripts to run all parameter searches, but they are completely analogous to this code cell. Using this strategy for all parameters, we have found the following hyperparameter values:

In [ ]:
training_params = TrainingHyperParams(
    batch_size=256,
    epochs=ALLOWED_EPOCHS_TO_TRAIN,
    steps_per_epoch=400*256,
    max_buffer_size=10*400*256
)
# TODO Update model params.
model_params = ModelHyperParams(
    initial_epsilon=0.5,
    epsilon_decay_constant=0.7,
    no_hidden_layers=2,
    units_per_hidden_layer=24,
    optimizer="adam",
    kernel_initializer="he_uniform"
)

To make this a bit more specific, let us use this agent to control the cart-pole for some episodes and measure for how long it achieves to keep the pole stable. For comparison we then repeat the same test using the `RandomAgent`. Note, that for the presentation in this Jupyter-Notebook we have disable a graphical rendering, but it can be switched on with the `show_graphics` flag, when run through a terminal. For convenience, we show here are two sample gifs for the two agents. $\textcolor{red}{\text{paste in gifs}}$ **[See here](https://stackoverflow.com/questions/51527868/how-do-i-embed-a-gif-in-jupyter-notebook) on how to embed gifs in jupyter notebooks [and here](https://stackoverflow.com/questions/54100582/saving-a-video-gif-file-for-the-open-ai-gymtaxi-environment) on how to make gifs from Gym.**

In [ ]:
environment_handler = CartPoleHandler(no_of_benchmark_episodes=5, show_graphics=False)
# TODO Adjust to env-handler, when code change is made.
q_learner = QLearner(env_handler, model_params)
# TODO Uncomment when ready.
#q_learner.load_weights("blabla")
# TODO Adjust to env-handler, when code change is made.
random_agent = RandomAgent(env_handler)

random_agent_median_episode_length = environment_handler.benchmark_agent(random_agent)
q_learner_median_episode_length = environment_handler.benchmark_agent(q_learner)

print("Random agent median epsiode length: ", random_agent_median_episode_length)
print("Q Learner median epsiode length: ", q_learner_median_episode_length)

## Model Comparison
Now that all different algorithms converge, we should compare them. That is we want to find out how fast and stable they converge to a solution compared to each other. A good metric to compare how the models perform after each epoch, is to train all four models $n$ times. After each training-epoch we then let the model control the cart-pole for $m$ episodes and count how many actions each of these episodes lasted.

After this, for each of the four models and each training epoch, we are left with $n \cdot m$ numbers representing the distribution of how long the episodes lasted. We can combine this into a single median and 95%-quantiles $\textcolor{red}{\text{check!!!}}$ and plot them over the epochs to compare the models. Note, however, that this combines two averaging processes: One over the $n$ independently trained agents and another over the $m$ episodes we ran after a given epoch for each competing model.

In [ ]:
# We can do the following just for 2 agents each and only 5 epochs or so, so that we can create all the logic
# and make sure it works. We can then run it again once all hyperparams are fixed.

# What should go into a separate script:
# Think about for how many epochs I want to allow models to train in the following? For sure more than 10.
# For each model, we train it 10 times.
# For each training run and after each epoch, we run the model 10 times (and allow for longer episodes than 200)
# Once an agent reaches the maximum epoch number (not before), we save the results to a dictonary, where
# The keys are [model_type, agent_id, epoch_number] and the values are the lists with results.
# After each agent finishes we pickle this to not lose progress. 

# What goes here:
# We load the script.
# We join all the lists for the agent_id's, so that we have a dict [model_type, epoch_number] -> results_list
# We take the quantiles: [model_type, epoch_number] -> (0.05, 0.50, 0.95)
# Turn into pandas dataframe
# Plot. Either separately or together.